# 初级实践培训作业：CIFAR10图像分类

本任务基于MindSpore的API来实现LeNet模型，并使用高阶封装`Model`进行模型训练、评估和推理。

本次任务中共有处代码缺失，请根据提示补齐代码，保证代码跑通，模型完成训练及评估。

## 作业提交

请将补充好代码并保存运行结果的notebook，提交至大模型平台，并填写调查问卷进行登记。调查问卷链接：[Link](https://wj.qq.com/s2/13864952/l0qz/)

大模型平台作业提交指南会通过邮件和本脚本发送，学员也可再微信交流群中查收作业提交视频指南。

## 环境准备

1. 安装MindSpore 1.10

    请参考[此链接](https://gitee.com/mindspore/docs/tree/r1.10/install)，选择对应硬件的安装指南。

    如：希望在windows CPU上安装MindSpore，建议点击mindspore_cpu_win_install_pip.md查看安装指南

2. 安装download

    `pip install download`
    
3. 安装matplotlib

    `pip install matplotlib`

In [1]:
import mindspore
from mindspore import nn
from mindspore import dtype as mstype
from mindspore.common.initializer import Normal
from mindspore.dataset import Cifar10Dataset, vision, transforms
from mindspore.train import Model, CheckpointConfig, ModelCheckpoint, LossMonitor


## 下载并处理数据集

### 数据集加载

CIFAR-10数据集共有60000张32*32的彩色图像，分为10个类别，每类有6000张图，数据集一共有50000张训练图片和10000张评估图片。

首先，使用`download`库下载CIFAR10数据集并解压，后使用`mindspore.dataset.Cifar10Dataset`加载数据集。

In [2]:
import os
# Download data from open datasets
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/datasets/cifar-10-binary.tar.gz"
path = download(url, "./data", kind="tar.gz", replace=True)


file_sizes: 100%|████████████████████████████| 170M/170M [00:15<00:00, 11.2MB/s]
Extracting tar.gz file...
Successfully downloaded / unzipped to ./data


### 数据集处理

通常情况下，我们需要对数据进行数据增强，从而增强模型的泛化性。在对Cifar10数据集的处理中，我们进行了如下操作：

- 仅针对训练数据集
    - 对图像数据进行随机裁剪（已实现）
    - 对图像数据进行随机水平翻转（已实现）
- 针对训练数据集和测试数据集
    - 使用`vision.Resize`，将图片数据的大小变为32x32
    - 使用`vision.Rescale`，将图片数据的元素数值缩小为原本的1/255
    - 使用`vision.Normalize`将图片数据进行归一化（已实现）
    - 使用`vision.HWC2CHW`将图片数据的shape从(height, width, channel)变为(channel, height, width)
    - 使用`transforms.TypeCast`将标签数据类型转换为`mindspore.Int32`(已实现)
- 将如上数据变换通过`map`分别添加到图片数据和标签数据中
- 使用`batch`对数据集进行批处理

***练习一：请根据上述说明，补充数据预处理代码***

In [3]:
def datapipe(data_path, usage, resize, batch_size):
    """数据处理Pipeline

    Args:
        data_path: 数据集路径
        usage: 训练数据集或测试数据集，输入为'train'或者'test'
        resize: 图像输出尺寸大小
        batch_size: batch大小
    """

    # 加载数据集
    dataset = Cifar10Dataset(data_path, usage)

    # 图像数据变换
    image_trans = []

    if usage == "train":
        image_trans += [
            vision.RandomCrop((32, 32), (4, 4, 4, 4)),  # 随机裁剪
            vision.RandomHorizontalFlip(prob=0.5)  # 随机水平翻转
        ]

    image_trans += [
        # 练习一：请在此处补充Resize和Rescale代码
        vision.Resize((32, 32)),
        vision.Rescale(1.0 / 255.0, 0.0),
        vision.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),  # 归一化
        # 练习一：请在此处补充输入图像shape的转换代码
        vision.HWC2CHW()

    ]

    # 标签数据数据变换
    label_trans = transforms.TypeCast(mstype.int32)

    # 练习一：请在此处补充数据映射操作代码
    # 提示：数据集每列的名称，可以通过dataset.get_col_names()获取
    image, label = dataset.get_col_names()
    dataset = dataset.map(image_trans, image)
    dataset = dataset.map(label_trans, label)
    
    # 练习一：请在此处补充批量操作代码
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

***练习二：请使用`create_tuple_iterator`或`create_dict_iterator`，获取图像数据的shape。***

图像数据的shape应该是(batch_size, channel, height, width)格式，其中batch_size为批的大小，channel为通道数，height和width是设定的resize后输出图片大小尺寸。

注：Tensor的shape可以通过`Tensor.shape`获取

In [4]:
train_dataset = datapipe(os.path.join(path, 'cifar-10-batches-bin'), 'train', 32, 64)
test_dataset = datapipe(os.path.join(path, 'cifar-10-batches-bin'), 'test', 32, 64)

# 练习二：请打印出任意数据集中图像数据的shape
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape)
print(label.shape)

(64, 3, 32, 32)
(64,)


## 创建模型

本次练习中，将使用LetNet5模型进行图像分类，LetNet5的模型结构如下图所示：

![lenet5](https://bkimg.cdn.bcebos.com/pic/30adcbef76094b36acaf1cb689986bd98d1001e93716?x-bce-process=image/watermark,image_d2F0ZXIvYmFpa2U5Mg==,g_7,xp_5,yp_5/format,f_auto)

结构拆解为：
1. 卷积神经网络(`nn.Conv2d`)：输出channel数为6，卷积核大小为5x5，stride为1
2. 激活函数ReLU(`nn.ReLU`)
3. 最大池化层(`nn.MaxPool2d`)：池化核大小为2，stride为2
4. 卷积神经网络(`nn.Conv2d`)：输出channel数为16，卷积核大小为5x5，stride为1
5. 激活函数ReLU(`nn.ReLU`)
6. 最大池化层(`nn.MaxPool2d`)：池化核大小为2，stride为2
7. 使用`nn.Flatten`对输入第0维以外的维度进行展平操作
8. 全连接层(`nn.Dense`)：输出维度为120
9. 激活函数ReLU(`nn.ReLU`)
10. 全连接层(`nn.Dense`)：输出维度为84
11. 激活函数ReLU(`nn.ReLU`)
12. 全连接层(`nn.Dense`)：输出维度为分类数量

***练习三：请参考上述模型结构，补齐模型构建的代码。***

In [5]:
class LeNet5(nn.Cell):

    def __init__(self, num_class, num_channel):
        super(LeNet5, self).__init__()
        # 练习三：实例化神经网络层，设置状态；第一个卷积神经网络已给出示例
        # nn.Conv2d(in_channels=?, out_channels=?, kernel_size=?, stride=?, pad_mode='valid')
        # nn.MaxPool2d(kernel_size=?, stride=?)
        # nn.Dense(in_channels=?, out_channels=?, weight_init=Normal(0.02))

        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dense1 = nn.Dense(16*5*5, 120)
        self.dense2 = nn.Dense(120, 84)
        self.dense3 = nn.Dense(84, num_class)
        self.flatten = nn.Flatten()

    def construct(self, x):
        # 练习三：书写正向逻辑
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        #print(x.shape)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        return x

# 练习三：模型实例化，注意这里的num_channel应该是在练习二中打印出的channel数值
model = LeNet5(num_class=10, num_channel=3)

## 定义损失函数和优化器

要训练神经网络模型，需要定义损失函数和优化器函数。

- 损失函数这里使用交叉熵损失函数`CrossEntropyLoss`。
- 优化器这里使用`Momentum`。

In [6]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.Momentum(model.trainable_params(), learning_rate=0.01, momentum=0.9)

## 训练及保存模型

### Model基本介绍

[Model](https://www.mindspore.cn/docs/en/r1.10/api_python/mindspore/mindspore.Model.html#mindspore.Model)是MindSpore提供的高阶API，可以进行模型训练、评估和推理。其接口的常用参数如下：

- `network`：用于训练或推理的神经网络。
- `loss_fn`：所使用的损失函数。
- `optimizer`：所使用的优化器。
- `metrics`：用于模型评估的评价函数。
- `eval_network`：模型评估所使用的网络，未定义情况下，`Model`会使用`network`和`loss_fn`进行封装。

`Model`提供了以下接口用于模型训练、评估和推理：

- `fit`：边训练边评估模型。
- `train`：用于在训练集上进行模型训练。
- `eval`：用于在验证集上进行模型评估。
- `predict`：用于对输入的一组数据进行推理，输出预测结果。

### 使用Model接口

对于简单场景的神经网络，可以在定义`Model`时指定前向网络`network`、损失函数`loss_fn`、优化器`optimizer`和评价函数`metrics`。

### 定义Callback

在深度学习训练中，为及时掌握网络模型的训练状态、实时观察网络模型各参数的变化情况和实现训练过程中用户自定义的一次额操作，MindSpore提供了回调机制（Callback）来实现上述功能。

Callback回调机制一般用在网络模型训练过程`Model.train`中，MindSpore的`Model`会按照Callback列表`callbacks`顺序执行回调函数。

在本次任务中，开始训练之前，MindSpore需要提前声明网络模型在训练过程中是否需要保存中间过程和结果，因此使用`ModelCheckpoint`接口用于保存网络模型和参数，以便进行后续的Fine-tuning（微调）操作。

In [7]:
steps_per_epoch = train_dataset.get_dataset_size()

config = CheckpointConfig(save_checkpoint_steps=steps_per_epoch)
ckpt_callback = ModelCheckpoint(prefix="cifar10", directory="./checkpoint", config=config)

loss_callback = LossMonitor(steps_per_epoch)

***练习四：通过MindSpore提供的`model.fit`接口进行网络训练，并通过Callback保存`ckpt`，监控`loss`变化***

In [8]:
train_dataset = datapipe(os.path.join(path, 'cifar-10-batches-bin'), 'train', 32, 64)
test_dataset = datapipe(os.path.join(path, 'cifar-10-batches-bin'), 'test', 32, 64)

epochs = 10

# 练习四：实例化Model，并通过model.fit进行网络训练
trainer = Model(model, loss_fn=loss_fn,optimizer=optimizer, metrics={'accuracy'})
trainer.fit(epochs, train_dataset, test_dataset, callbacks=[ckpt_callback, loss_callback])

epoch: 1 step: 781, loss is 1.652235746383667
Eval result: epoch 1, metrics: {'accuracy': 0.4221754807692308}
epoch: 2 step: 781, loss is 1.5244983434677124
Eval result: epoch 2, metrics: {'accuracy': 0.495693108974359}
epoch: 3 step: 781, loss is 1.2929081916809082
Eval result: epoch 3, metrics: {'accuracy': 0.5512820512820513}
epoch: 4 step: 781, loss is 1.3503402471542358
Eval result: epoch 4, metrics: {'accuracy': 0.5614983974358975}
epoch: 5 step: 781, loss is 1.119059681892395
Eval result: epoch 5, metrics: {'accuracy': 0.5739182692307693}
epoch: 6 step: 781, loss is 1.2889869213104248
Eval result: epoch 6, metrics: {'accuracy': 0.5950520833333334}
epoch: 7 step: 781, loss is 1.3357627391815186
Eval result: epoch 7, metrics: {'accuracy': 0.5958533653846154}
epoch: 8 step: 781, loss is 1.2336413860321045
Eval result: epoch 8, metrics: {'accuracy': 0.6001602564102564}
epoch: 9 step: 781, loss is 1.24985671043396
Eval result: epoch 9, metrics: {'accuracy': 0.6185897435897436}
epoch:

训练过程中会打印loss值，loss值会波动，但总体来说loss值会逐步减小，精度逐步提高。每个人运行的loss值有一定随机性，不一定完全相同。

通过模型运行测试数据集得到的结果，验证模型的泛化能力：

1. 使用`model.eval`接口读入测试数据集。
2. 使用保存后的模型参数进行推理。

***练习五：通过`model.eval`打印训练好的模型在测试集上的准确率***

In [9]:
# 练习五：使用model.eval进行模型评估
acc = trainer.eval(test_dataset)
acc

{'accuracy': 0.6065705128205128}